# Unsupervised MERA

- MERA stands for Music Emotion Recognition Algorithm which was built to classify emotions from music based on both a song's lyrics and audio. The algorithm was built on the concepts of Valence and Arousal in music, which represent the spectrum of positive to negative emotions and low to high energy respectively, and Quadrants, which are representations of combinations of Valence and Arousal depending on their sign (-/+). This algorithm was a supervised task as we had a target variable.

- In this notebook, I'll be experimenting whether it's also possible to identify the target lables / Quadrants using unsupervied learning techniques. Since the dataset I'll be using for the experiment already has the expected outcome / Quadrants, I should be able to evaluate how well unsupervied techniques can be used in identitying the lables.